In [1]:
import torch
import torch.nn as nn
import torch.optim as optimize
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pandas as pd
from skimage import io
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
from sklearn.metrics import classification_report

In [2]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs, L1RegWeight):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            batches = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                batches += 1
                print("Batch " + str(batches))
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    if (phase != 'val' and L1RegWeight != 0):
                        loss += L1RegWeight * (sum(p.abs().sum() for p in model.parameters()))

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.detach().item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).detach()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [3]:
def doValidation(model, dataloader):
    results = {"real": [], "pred": []}
    model.eval()
    torch.set_grad_enabled(False)
    for inputs, labels in dataloader:
        for x in labels.tolist():
            results['real'].append(x)
        inputs = inputs.to(device)
        output = model(inputs)
        for x in torch.argmax(output, dim=1).tolist():
            results["pred"].append(x)
    return results

In [4]:
def initModel(num_classes, batchNormalization):
    if batchNormalization:
        model_ft = models.vgg16_bn(pretrained=True)
    else:
        model_ft = models.vgg16(pretrained=True)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    return model_ft

In [5]:
modelTest = initModel(3, True)

In [6]:
lr = 0.001
momentum = 0.9
regL2_mult = 0.0005

In [7]:
criterion = nn.CrossEntropyLoss()
optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor()
    ]),
}

In [9]:
class DisastersDatasets(Dataset):
    def __init__(self, csv, root_dir, transform=None):
        self.csv = pd.read_csv(csv, delim_whitespace=True, header=None, usecols=[0,1], names=["img","lbl"])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.csv["img"][idx])

        image = io.imread(img_name)
        image = Image.fromarray(image)
        
        if self.transform:
            image = self.transform(image)
        label = self.csv["lbl"][idx]

        if (image.shape[0] == 4):
            image = image[0:3]

        return image, label

In [10]:
goal_disaster_train_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\matthew.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
test_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\matthew.test", "ASONAM17_Damage_Image_Dataset", data_transforms['val'])
dev_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\matthew.dev", "ASONAM17_Damage_Image_Dataset", data_transforms['val'])
other_disaster_train_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\ruby.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
google_train_set = DisastersDatasets("ASONAM17_Damage_Image_Dataset\\gg.train", "ASONAM17_Damage_Image_Dataset", data_transforms['train'])
concatenatedTrainSet = ConcatDataset([goal_disaster_train_set, other_disaster_train_set, google_train_set])

In [11]:
concatenated_train_loader = DataLoader(concatenatedTrainSet, batch_size = 14, shuffle = True)
print("concateSize " + str(len(concatenated_train_loader)))
goal_disaster_train_loader = DataLoader(goal_disaster_train_set, batch_size = 14, shuffle = True)
print("Goalsize " + str(len(goal_disaster_train_loader)))
other_disaster_train_loader = DataLoader(other_disaster_train_set, batch_size = 14, shuffle=True)
print("othersize " + str(len(other_disaster_train_loader)))
google_train_loader = DataLoader(google_train_set, batch_size = 14, shuffle=True)
print("Googlesize " + str(len(google_train_loader)))
test_loader = DataLoader(test_set, batch_size = 14, shuffle=True)
dev_loader = DataLoader(dev_set, batch_size = 14, shuffle=True)

concateSize 191
Goalsize 26
othersize 36
Googlesize 129


In [12]:
device = torch.device("cuda:0")
modelTest.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [13]:
doSeparate = True
history = []
if doSeparate:
    for x in range(0, 3):
        torch.cuda.empty_cache()
        modelTest, history_temp = train_model(modelTest, {"train": google_train_loader, "val": test_loader}, criterion, optim, 3, 0)
        history.append(history_temp)
        lr = lr/10
        optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
    lr = 0.001
    for x in range(0, 3):
        torch.cuda.empty_cache()
        modelTest, history_temp = train_model(modelTest, {"train": other_disaster_train_loader, "val": test_loader}, criterion, optim, 3, 0)
        history.append(history_temp)
        lr = lr/10
        optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
    lr = 0.001
    for x in range(0, 3):
        torch.cuda.empty_cache()
        modelTest, history_temp = train_model(modelTest, {"train": goal_disaster_train_loader, "val": test_loader}, criterion, optim, 3, 0)
        history.append(history_temp)
        lr = lr/10
        optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
else:
    for x in range(0, 3):
        torch.cuda.empty_cache()
        modelTest, history_temp = train_model(modelTest, {"train": concatenated_train_loader, "val": test_loader}, criterion, optim, 3, 0)
        history.append(history_temp)
        lr = lr/10
        optim = optimize.SGD(modelTest.parameters(), lr=lr, momentum=momentum, weight_decay=regL2_mult)
torch.cuda.empty_cache()

Epoch 0/2
----------
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch

In [14]:
results = doValidation(modelTest, dev_loader)
targetNames = ['none', 'mild', 'severe']
print(results['real'])
print(results['pred'])
print(classification_report(results['real'], results['pred'], target_names=targetNames))

[1, 1, 0, 2, 0, 0, 0, 0, 2, 1, 1, 0, 1, 2, 1, 2, 1, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 1, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0, 0]
[0, 1, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 0, 0, 2, 0, 2, 2, 0, 1]
              precision    recall  f1-score   support

        none       0.78      0.86      0.82        66
        mild       0.48      0.43      0.45        28
      severe       0.64      0.54      0.58        26

    accuracy                           0.69       120
   mac